# Image Classifier Model
Brandon Welsh

4/11/2024

--------------------
## Data Loading

In [1]:
# Import all required dependencies in this notebook
import pandas as pd
import requests
from PIL import Image
from io import BytesIO

In [2]:
# Define a dataframe, load the csv
df = pd.read_csv('./Resources/image-Sentiment-polarity-DFE.csv')

In [3]:
# View the first and last 5 columns of the dataframe
df

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,which_of_these_sentiment_scores_does_the_above_image_fit_into_best,which_of_these_sentiment_scores_does_the_above_image_fit_into_best:confidence,imageurl,which_of_these_sentiment_scores_does_the_above_image_fit_into_best_gold,which_of_these_sentiment_scores_does_the_above_image_fit_into_best_gold_reason
0,694550568,True,golden,47,NaN,Positive,0.6395,http://farm1.staticflickr.com/20/71235476_3103...,Highly positive\nPositive,"He is smiling. => happy, joy"
1,694550569,True,golden,36,NaN,Positive,0.6458,http://farm4.staticflickr.com/3096/2584485524_...,Highly positive\nPositive,A beautiful scenery. => serenity
2,694550570,True,golden,35,NaN,Negative,0.8420,http://farm3.staticflickr.com/2880/11733997835...,Negative\nHighly negative,"They look serious. => sadness, fear"
3,694550571,True,golden,41,NaN,Positive,0.7216,http://farm4.staticflickr.com/3022/2286937150_...,Highly positive\nPositive,"A cute puppy. => happy, joy"
4,694550572,True,golden,46,NaN,Positive,0.8801,http://farm1.staticflickr.com/163/357381033_d0...,Highly positive\nPositive,"She looks fun. => fun, joy, happy"
...,...,...,...,...,...,...,...,...,...,...
15608,694566363,False,finalized,3,3/26/15 1:59,Positive,0.6705,http://farm3.static.flickr.com/2198/1572103607...,NaN,NaN
15609,694566364,False,finalized,3,3/26/15 2:13,Positive,0.3373,http://farm1.static.flickr.com/100/263134442_6...,NaN,NaN
15610,694566365,False,finalized,3,3/26/15 2:54,Positive,0.6663,http://farm2.static.flickr.com/1360/1392747419...,NaN,NaN
15611,694566366,False,finalized,3,3/26/15 2:46,Negative,0.6760,http://farm3.static.flickr.com/2077/1925076121...,NaN,NaN


-------------------
# Data Cleaning

In [4]:
# Rename some of the poorly named columns
df = df.rename(columns={
    'which_of_these_sentiment_scores_does_the_above_image_fit_into_best': 'sentiment',
    'which_of_these_sentiment_scores_does_the_above_image_fit_into_best:confidence': 'confidence',
    'which_of_these_sentiment_scores_does_the_above_image_fit_into_best_gold_reason': 'reason',
})

In [5]:
# Check if that worked
df.head(1)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,sentiment,confidence,imageurl,which_of_these_sentiment_scores_does_the_above_image_fit_into_best_gold,reason
0,694550568,True,golden,47,NaN,Positive,0.6395,http://farm1.staticflickr.com/20/71235476_3103...,Highly positive\nPositive,"He is smiling. => happy, joy"


In [7]:
# Define a function to download and resize an image
def download_and_resize(url, size=(128, 128)):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for 4xx/5xx status codes
        img = Image.open(BytesIO(response.content))
        img = img.resize(size)
        return img
    except requests.exceptions.RequestException:
        print(f"Failed to download {url}")
        return None

In [8]:
# Apply the function to the 'imageurl' column
df['image'] = df['imageurl'].apply(download_and_resize)

Failed to download http://farm1.staticflickr.com/163/357381033_d0176b7f37.jpg
Failed to download http://farm5.staticflickr.com/4102/4889703839_fd61de8a7a.jpg
Failed to download http://farm6.staticflickr.com/5208/5334167439_6ced868815.jpg
Failed to download http://farm4.staticflickr.com/3732/9496265700_37643409c4.jpg
Failed to download http://farm2.staticflickr.com/1346/5141227803_dc88f098fe.jpg
Failed to download http://farm1.staticflickr.com/49/157202754_bf1696894d.jpg
Failed to download http://farm6.staticflickr.com/5516/9080155048_a549c09d12.jpg
Failed to download http://farm4.staticflickr.com/3139/3004924293_c101edb44d.jpg
Failed to download http://farm4.staticflickr.com/3687/11445715234_9bac7dfbae.jpg
Failed to download http://farm4.staticflickr.com/3157/2645971210_a7928781a1.jpg
Failed to download http://farm6.staticflickr.com/5484/10640400663_e811f54c5d.jpg
Failed to download http://farm3.staticflickr.com/2585/4144135046_6cbeccc4c0.jpg
Failed to download http://farm4.staticflick

KeyboardInterrupt: 

In [ ]:
# Filter out rows where 'image' is None
df = df[df['image'].notna()]